# Final Project
## Data 311
## Phelix Tang, Shintaro Osuga, Spencer Hunt
## NBA Rookie Salaries

For our final project, we're building a model, to predict a player's first year salary in the NBA, based on their college statistics

In [4]:
import requests
import bs4
import re
import pandas as pd
import time
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import DataFrame
import math
from numpy import mean
from numpy import std



# Part 1 Preprocessing and Cleaning
So for our datasets we have two, one is NBA salaries, and the other is college statistics, we're planning on merging them by player name. Our NBA salaries consist of an additional two datasets, with one containing names and the other with player id. We also import and inflation table and use that to adjust the salaries.

In [5]:
salary = pd.read_csv('https://query.data.world/s/4qf5meavexqlu677lpr76gbbvnxy7g')
salary2 = pd.read_csv('https://query.data.world/s/b6xcgntwji5kqi75k2twovrdrjxeoq')

In [6]:
player = pd.read_csv("players.csv")
inflation = pd.read_csv('inflation.csv')
salary.rename({'player_id':'id'}, axis='columns', inplace=True)
salary2.rename({'_id':'id'}, axis='columns', inplace=True)


In [7]:
player

,Unnamed: 0,active_from,active_to,birth_date,college,height,name,position,url,weight,...,NCAA__3ptpg,NCAA_efgpct,NCAA_fgapg,NCAA_fgpct,NCAA_fgpg,NCAA_ft,NCAA_ftapg,NCAA_ftpg,NCAA_games,NCAA_ppg
0,0,1991,1995,"June 24, 1968",Duke University,6-10,Alaa Abdelnaby,F-C,/players/a/abdelal01.html,240.0,...,0.0,NaN,5.6,0.599,3.3,0.728,2.5,1.8,134.0,8.5
1,1,1969,1978,"April 7, 1946",Iowa State University,6-9,Zaid Abdul-Aziz,C-F,/players/a/abdulza01.html,235.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1970,1989,"April 16, 1947","University of California, Los Angeles",7-2,Kareem Abdul-Jabbar,C,/players/a/abdulka01.html,225.0,...,NaN,NaN,16.8,0.639,10.7,0.628,7.9,5.0,88.0,26.4
3,3,1991,2001,"March 9, 1969",Louisiana State University,6-1,Mahmoud Abdul-Rauf,G,/players/a/abdulma02.html,162.0,...,2.7,NaN,21.9,0.474,10.4,0.863,6.4,5.5,64.0,29.0
4,4,1998,2003,"November 3, 1974","University of Michigan, San Jose State University",6-6,Tariq Abdul-Wahad,F,/players/a/abdulta01.html,223.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4571,4571,2018,2018,"January 4, 1997",NaN,6-11,Ante Zizic,F-C,/players/z/zizican01.html,250.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4572,4572,1983,1983,"December 20, 1953",Kent State University,7-1,Jim Zoet,C,/players/z/zoetji01.html,240.0,...,NaN,NaN,2.9,0.476,1.4,0.429,1.0,0.4,63.0,3.2
4573,4573,1971,1971,"June 7, 1948",Duquesne University,6-1,Bill Zopf,G,/players/z/zopfbi01.html,170.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4574,4574,2017,2018,"March 18, 1997",NaN,7-1,Ivica Zubac,C,/players/z/zubaciv01.html,265.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
salary

,league,id,salary,season,season_end,season_start,team
0,NBA,abdelal01,395000,1990-91,1991,1990,Portland Trail Blazers
1,NBA,abdelal01,494000,1991-92,1992,1991,Portland Trail Blazers
2,NBA,abdelal01,500000,1992-93,1993,1992,Boston Celtics
3,NBA,abdelal01,805000,1993-94,1994,1993,Boston Celtics
4,NBA,abdelal01,650000,1994-95,1995,1994,Sacramento Kings
...,...,...,...,...,...,...,...
14158,NBA,zipsepa01,750000,2016-17,2017,2016,Chicago Bulls
14159,NBA,zipsepa01,1312611,2017-18,2018,2017,Chicago Bulls
14160,NBA,zizican01,1645200,2017-18,2018,2017,Cleveland Cavaliers
14161,NBA,zubaciv01,1034956,2016-17,2017,2016,Los Angeles Lakers


### 1.1 Cleaning the Salary column
Since we have multiple dataframes, we have to combine all of them into something useable, here we're merging our two salary dataframe. We also map our inflation by year into our all_merge df. Our Inflation dataframe has a column of years, which map to a value that is the inflation rate towards the current year, we then multiply all the salaries by that inflation rate. We also map the player's id to the player's name for convience

In [9]:
merge_salary = salary2.merge(salary, on='id')
all_merge = player.merge(merge_salary, on='name')
all_merge = all_merge.merge(inflation, left_on='season_start', right_on='2020')
all_merge['salary'] = all_merge['1.0123'] * all_merge['salary']
all_merge

,Unnamed: 0,active_from,active_to,birth_date,college_x,height_x,name,position_x,url,weight_x,...,weight_y,league,salary,season,season_end,season_start,team,2020,1.23 %,1.0123
0,0,1991,1995,"June 24, 1968",Duke University,6-10,Alaa Abdelnaby,F-C,/players/a/abdelal01.html,240.0,...,240lb,NBA,800270.0,1990-91,1991,1990,Portland Trail Blazers,1990,5.40 %,2.026
1,3,1991,2001,"March 9, 1969",Louisiana State University,6-1,Mahmoud Abdul-Rauf,G,/players/a/abdulma02.html,162.0,...,162lb,NBA,3363160.0,1990-91,1991,1990,Denver Nuggets,1990,5.40 %,2.026
2,12,1988,1993,"November 15, 1962",Oral Roberts University,6-11,Mark Acres,F-C,/players/a/acresma01.html,220.0,...,220lb,NBA,885362.0,1990-91,1991,1990,Orlando Magic,1990,5.40 %,2.026
3,20,1986,1996,"January 19, 1963",Boston College,5-10,Michael Adams,G,/players/a/adamsmi01.html,162.0,...,162lb,NBA,1671450.0,1990-91,1991,1990,Denver Nuggets,1990,5.40 %,2.026
4,28,1982,1994,"December 10, 1959",DePaul University,6-6,Mark Aguirre,F-G,/players/a/aguirma01.html,232.0,...,232lb,NBA,2258990.0,1990-91,1991,1990,Detroit Pistons,1990,5.40 %,2.026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14425,4053,1986,1997,"June 9, 1964",University of Oklahoma,6-9,Wayman Tisdale,F-C,/players/t/tisdawa01.html,240.0,...,240lb,NBA,5310000.0,1989-90,1990,1989,Sacramento Kings,1989,4.83 %,2.124
14426,4086,1982,1991,"February 16, 1959",University of Notre Dame,6-6,Kelly Tripucka,F-G,/players/t/tripuke01.html,220.0,...,220lb,NBA,2124000.0,1989-90,1990,1989,Charlotte Hornets,1989,4.83 %,2.124
14427,4370,1983,1999,"January 12, 1960",University of Georgia,6-7,Dominique Wilkins,F-G,/players/w/wilkido01.html,200.0,...,215lb,NBA,4460400.0,1989-90,1990,1989,Atlanta Hawks,1989,4.83 %,2.124
14428,4384,1982,1998,"March 8, 1960",University of Maryland,6-8,Buck Williams,F-C,/players/w/willibu01.html,215.0,...,215lb,NBA,2761200.0,1989-90,1990,1989,Portland Trail Blazers,1989,4.83 %,2.124


### 1.2 Getting rid of player's without college stats
Since we're training our model off college stats, if an NBA player went straight to the league, we'll just completely cut them off our dataframe.

In [10]:
tester = all_merge
tester = tester[tester['college_x'].notna()]
tester = tester.replace(np.nan, '0')

### 1.3 Grabbing First Year Salaries
Our NBA dataframe has several rows for players, who've played several years in the NBA, in this block, we're grabbing each indivual player, and just grabbing their first year in the NBA, Since we're just trying to predict Rookie year salary. Below this, we also convert heights which is in feet/inches to purely inches, just for easier computational work.

In [11]:
test = pd.DataFrame()
for x in tester['name'].unique():
    test = test.append(tester.loc[tester['name']==x].sort_values('season', ascending=True).head(1))
    
test = test.reset_index()

In [12]:
def convert(s):
    split = s.split('-')
    inches = (float(split[0]) * 12) + float(split[1])
    return inches

for x in test.index:
    test["height_x"][x] = convert(test["height_x"][x])

/home/hunts6/311/data311_env/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


### 1.4 Cleaning season year
The remaining row with a problematic data type is the season column, with there being a - inbetween the years and having two year dates. We decided to drop the ending year and keep the beginning year.

In [13]:
test['season'] = test['season'][:].str.slice(stop=4)

### 1.5 Grabbing just the Columns we need and Type Conversion.
Now that we have our full dataset, We can just grab the columns that play a role into their game performance, these are stats such as, height, position, field goal percentage, etc. We next, convert all of these columns into floats, so that they can be used in our training model.

In [14]:
all_merge = test

all_merge = all_merge[["college_x","height_x","position_x","NCAA__3ptapg","NCAA__3ptpct","NCAA__3ptpg","NCAA_fgapg",
                        "NCAA_fgpct","NCAA_fgpg","NCAA_ft","NCAA_ftapg","NCAA_ftpg","NCAA_games","NCAA_ppg","salary","season","draft_round","draft_pick"]]
print(all_merge.to_string())

                                                                                                                       college_x height_x position_x NCAA__3ptapg NCAA__3ptpct NCAA__3ptpg NCAA_fgapg NCAA_fgpct NCAA_fgpg NCAA_ft NCAA_ftapg NCAA_ftpg NCAA_games NCAA_ppg       salary season draft_round     draft_pick
0                                                                                                                Duke University       82        F-C            0            0           0        5.6      0.599       3.3   0.728        2.5       1.8        134      8.5   800270.000   1990   1st round   25th overall
1                                                                                                     Louisiana State University       73          G          7.2        0.372         2.7       21.9      0.474      10.4   0.863        6.4       5.5         64       29  3363160.000   1990   1st round    3rd overall
2                                                      

In [15]:
for columns in all_merge.columns[3:15]:
    all_merge[columns]=all_merge[columns].astype(float)
test["height_x"] = test["height_x"].astype(float)

/home/hunts6/311/data311_env/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### 1.6 Getting rid of outliers
We have a bunch of players who only made around $8,000 or so, accounting for inflation, these are huge outliers since the average nba salary is just above 900k a year, we decided to drop the first 250, lowest salaries. Which stops at around 100k.

In [16]:

test = all_merge
test = test.sort_values(by="salary", ascending = True)
test = test.iloc[250:]

### 1.7 Cleaning Draft round and Draft pick columns
Here we simply clean our draft round and draft pick so that they can be inputted into our training model.

In [17]:
test['draft_round'] = re.findall(r'\d+', test['draft_round'].to_string())[1::2]
test['draft_pick'] = re.findall(r'\d+',test['draft_pick'].to_string())[1::2]
test[['draft_round','draft_pick']]

,draft_round,draft_pick
445,2,39
323,2,46
180,3,70
99,0,0
441,5,100
...,...,...
1876,1,1
1854,1,1
156,1,1
1915,1,2


### Putting it in a csv
We store our fully clean and merged data as a CSV called Processed.csv

In [18]:
Processed = pd.DataFrame()
Processed = test
Processed.to_csv('Processed.csv', index=False, header=Processed.columns)

# Conclusion
We merged all of our datasets, and mapped our player's to their respecitive rows, from there we cleaned all the columns that we would be using for our training model, we also replaced NaN values for players who didn't have their shooting scores. We then proceeded to drop some outliers, and converted everything to numerical values, we're left with 3 categorical columns which will be encoded to numerical values in our next notebook.